In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import random
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

slim = tf.contrib.slim

losses = tf.contrib.losses

input_img = tf.placeholder(tf.float32, shape=[None, 784])

output = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(input_img, [-1,28,28,1])

is_training = tf.placeholder(tf.bool)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
net ={}
net['Conv1'] = slim.conv2d(x_image, 32, [5,5])
net['MaxPool1'] = slim.max_pool2d(net['Conv1'], [2,2])
net['Conv2'] = slim.conv2d(net['MaxPool1'], 64, [5,5])
net['MaxPool2'] = slim.max_pool2d(net['Conv2'], [2,2])
net['flatten'] = slim.flatten(net['MaxPool2']) #NoFc1 earlier
net['Fc1'] = slim.fully_connected(net['flatten'], 1024)
net['dropout'] = slim.dropout(net['Fc1'], 0.5, is_training=is_training)
net['output'] = slim.fully_connected(net['dropout'], 10, activation_fn=None)

In [4]:
cross_entropy_loss = tf.reduce_mean(losses.softmax_cross_entropy(net['output'], output))

correct_prediction = tf.equal(tf.argmax(net['output'], 1), tf.argmax(output, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

epochs = 200

initial_acc = 0.0
variables_to_restore = slim.get_model_variables()
restore = tf.train.Saver(variables_to_restore)
print(tf.train.latest_checkpoint('./'))
with tf .Session() as sess:
    restore.restore(sess, './Mnist_NLA_rep2.ckpt')
    test_accuracy = accuracy.eval(feed_dict={input_img:mnist.test.images[:100], output:mnist.test.labels[:100], 
                                             is_training:False})
    print("The test accuracy is:", test_accuracy*100, "%")
    conv1_act_arr = net['Conv1'].eval(feed_dict={input_img:mnist.train.images[:2000], output:mnist.train.labels[:2000], 
                                                is_training:False})
    conv2_act_arr = net['Conv2'].eval(feed_dict={input_img:mnist.train.images[:2000], output:mnist.train.labels[:2000], 
                                                is_training:False})
    np.save("./data_rep/Conv1_Activations_2000_bad", conv1_act_arr)
    np.save("./data_rep/Conv2_Activations_2000_bad", conv2_act_arr)
    


./Mnist_NLA_rep2.ckpt
The test accuracy is: 93.9999997616 %


In [ ]:
print(mnist.train.images.shape)